#### Importing Libraries:

In [1]:
import os, sys
import pandas as pd
import numpy as np
import cv2
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
import random
import pickle

#### Defining Targets:

In [2]:
base_dir = "./ranzcr_clip_catheter_line_classification/"
df = pd.read_csv(os.path.join(base_dir, "train.csv"))
# df = df.drop(['StudyInstanceUID', 'PatientID'], axis=1)
df

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0,0,0,0,0,0,1,0,0,0,0,ec89415d1
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0,0,1,0,0,1,0,0,0,1,0,bf4c6da3c
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0,0,0,0,0,0,0,0,1,0,0,3fc1c97e5
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0,0,0,0,0,0,0,1,0,0,0,c31019814
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0,0,0,0,0,0,0,0,0,1,0,207685cd1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30078,1.2.826.0.1.3680043.8.498.74257566841157531124...,0,0,1,0,0,0,0,0,1,1,0,5b5b9ac30
30079,1.2.826.0.1.3680043.8.498.46510939987173529969...,0,0,0,0,0,0,0,0,0,1,0,7192404d8
30080,1.2.826.0.1.3680043.8.498.43173270582850645437...,0,0,1,0,0,1,0,1,0,1,0,d4d1b066d
30081,1.2.826.0.1.3680043.8.498.95092491950130838685...,0,0,0,0,0,0,0,0,1,0,0,01a6602b8


#### Tackle data consistency:

In [3]:
#The data at hand includes:

my_df = pd.read_csv(os.path.join(base_dir, "my_data.csv"), header = None)
my_df

FileNotFoundError: [Errno 2] No such file or directory: './ranzcr_clip_catheter_line_classification/my_data.csv'

In [ ]:
#deleting the extention ".jpg" from "my_data":

for i in range(0, len(my_df)):
    my_df[0][i] = my_df[0][i][:-4]

In [ ]:
#finding the matched data:

temp_arr = df['StudyInstanceUID'].values.tolist()
lst_index = []

for i in range(0, len(my_df)):
    lst_index.append(temp_arr.index(my_df[0][i]))
    
# #A slower method of O(n^2):    
# for i in range(0, len(my_df)): #or using .find() method
#     for j in range(0, len(df)):
#         if my_df[0][0] == df['StudyInstanceUID'][j]:
#             lst_index.append(j)    
    
lst_index.sort()
# lst_index

In [ ]:
#either deletes the excess data from the original dataset, or sorts them.

labels = df.iloc[lst_index, :]
labels

In [ ]:
#solving data leackage issue:

temp_arr = labels['PatientID'].values.tolist()
temp_lst = []
lst_id = []

for i in range(0, len(labels)):
    for j in range(i+1, len(labels)):
        if temp_arr[i] == temp_arr[j]:
            temp_lst.append(j) #labels.index[j]
    if temp_lst != []:
        temp_lst.insert(0, i) #labels.index[i]
        lst_id.append(temp_lst)
        temp_lst = []

# len(lst_id) #verified!
# lst_id
arr_id = np.array(lst_id)
arr_id

In [ ]:
#dropping unnecessary columns in the df "labels":

labels = labels.drop(['StudyInstanceUID', 'PatientID'], axis=1)
# labels

In [ ]:
arr_labels = labels.values
arr_labels

In [ ]:
arr_labels.shape

#### Resizing images:

In [ ]:
def load_resize_training_data(imageSize = 224, numSamples = 40000): #all image sizes: 650*650 #also changed from 10000 to 1000 due to my system memory.
    path = os.path.join(base_dir, "train/")
    trainingData = []
    i=0
    for img in os.listdir(path):
        if i == numSamples:
            break
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE) #plt.imshow will not result in a very fair respresentation.
        resized_array = cv2.resize(img_array, (imageSize, imageSize))
        trainingData.append(resized_array) #train_df.at[i, 'combined_label']]
        i += 1

#     plt.imshow(trainingData[0], cmap = "gray")

    return trainingData

data_train = load_resize_training_data()
data_train

In [ ]:
len(data_train) #verified!

In [ ]:
data_train[0].shape #verified!

In [ ]:
arr_train = np.array(data_train)

In [ ]:
arr_train.shape

In [ ]:
#So far, we have the x-ray images in the "arr_train" variable. -> saving to "train.pickle"
#And also the 11 labels in the "arr_labels" variable. -> saving to "target.pickle"
#And all the repeted PatientIDs in the "arr_id" variable. -> saving to "id.pickle"

#### Saving the files:

In [ ]:
#save x data
pickle_out = open("train.pickle", "wb")
pickle.dump(arr_train, pickle_out)
pickle_out.close()
#save y data
pickle_out = open("target.pickle", "wb")
pickle.dump(arr_labels, pickle_out)
pickle_out.close()
#save PatientIds:
pickle_out = open("id.pickle", "wb")
pickle.dump(arr_id, pickle_out)
pickle_out.close()